In [ ]:

import os
import re
import glob
import math
from datetime import datetime

import pandas as pd
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import joblib

DATA_DIR = r"Dataset"  
OUTPUT_CSV = "case_ranking.csv"
MODEL_PATH = "case_influence_rf.joblib"
CURRENT_YEAR = datetime.now().year


In [ ]:
import os
import re
import json
from bs4 import BeautifulSoup

CURRENT_YEAR = 2025

def parse_case_xmls(DATA_DIR, keywords=None, dump_json=True):
    """
    Parse all XML case files in a folder and return a list of extracted dicts.
    Also prints progress and count of parsed files.
    """
    if keywords is None:
        keywords = [
            "declaratory relief", "injunctive relief", "injunction",
            "binding precedent", "precedent", "stare decisis",
            "declaration", "section", "s ", "act", "order"
        ]

    results = []
    count = 0

    for filename in os.listdir(DATA_DIR):
        if not filename.lower().endswith(".xml"):
            continue  # skip non-xml files

        filepath = os.path.join(DATA_DIR, filename)

        try:
            with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
                raw = f.read()

            soup = BeautifulSoup(raw, "xml")

            # Case name
            name_tag = soup.find("name")
            case_name = name_tag.get_text(strip=True) if name_tag else filename

            # Catchphrases
            catchphrases = []
            for cp in soup.find_all(re.compile("^catchphrase$", re.I)):
                txt = cp.get_text(" ", strip=True)
                if txt:
                    catchphrases.append(txt)

            # Sentences
            sentences = []
            for s in soup.find_all(re.compile("^sentence$", re.I)):
                txt = s.get_text(" ", strip=True)
                if txt:
                    sentences.append(txt)

            full_text = " ".join(sentences).strip()

            # Feature counts
            sentence_count = len(sentences)
            word_count = len(full_text.split())
            catchphrase_count = len(catchphrases)

            # Year + court code
            year = None
            court_code = None
            m = re.search(r"\[(\d{4})\]\s*([A-Z]{1,5})", case_name)
            if m:
                year = int(m.group(1))
                court_code = m.group(2)
            else:
                m2 = re.search(r"(\d{4})", case_name)
                if m2:
                    year = int(m2.group(1))

            if year is None:
                year = CURRENT_YEAR

            court_code_upper = (court_code or "").upper()
            court_level_map = {"HCA": 3, "HIG": 3, "FCA": 2, "HCA:": 3}
            court_level = court_level_map.get(court_code_upper, 1)

            # Keywords
            lc_text = full_text.lower()
            keyword_count = sum(lc_text.count(k.lower()) for k in keywords)

            # Recency
            recency_years = CURRENT_YEAR - year

            result = {
                "file": filename,
                "case_name": case_name,
                "catchphrases_list": catchphrases,
                "catchphrase_count": catchphrase_count,
                "sentences": sentence_count,
                "word_count": word_count,
                "full_text": full_text,
                "year": year,
                "court_code": court_code_upper if court_code_upper else None,
                "court_level": court_level,
                "keyword_count": keyword_count,
                "recency_years": recency_years
            }

            results.append(result)
            count += 1
            print(f"✅ Parsed {count}: {filename}")

            # Dump JSON (optional)
            if dump_json:
                out_file = os.path.join(DATA_DIR, filename.replace(".xml", ".json"))
                with open(out_file, "w", encoding="utf-8") as jf:
                    json.dump(result, jf, indent=4, ensure_ascii=False)

        except Exception as e:
            print(f"❌ Error parsing {filename}: {e}")

    print(f"\n📊 Total XML files parsed: {count}")
    return results


In [7]:
# Call the parser on your data folder
cases = parse_case_xmls(DATA_DIR)

# Save everything into one CSV (optional)
import pandas as pd
df = pd.DataFrame(cases)
df.to_csv("Parsed_cases.csv", index=False)

print("✅ Parsing complete, CSV saved as cases_parsed.csv")
# parses all .xml files in "data/" folder


✅ Parsed 1: 06_1.xml
✅ Parsed 2: 06_100.xml
✅ Parsed 3: 06_1001.xml
✅ Parsed 4: 06_1004.xml
✅ Parsed 5: 06_1005.xml
✅ Parsed 6: 06_1006.xml
✅ Parsed 7: 06_1015.xml
✅ Parsed 8: 06_1017.xml
✅ Parsed 9: 06_1018.xml
✅ Parsed 10: 06_1019.xml
✅ Parsed 11: 06_102.xml
✅ Parsed 12: 06_1021.xml
✅ Parsed 13: 06_1022.xml
✅ Parsed 14: 06_1023.xml
✅ Parsed 15: 06_1026.xml
✅ Parsed 16: 06_1027.xml
✅ Parsed 17: 06_1028.xml
✅ Parsed 18: 06_1029.xml
✅ Parsed 19: 06_1032.xml
✅ Parsed 20: 06_1033.xml
✅ Parsed 21: 06_1041.xml
✅ Parsed 22: 06_1042.xml
✅ Parsed 23: 06_1043.xml
✅ Parsed 24: 06_1044.xml
✅ Parsed 25: 06_1045.xml
✅ Parsed 26: 06_1046.xml
✅ Parsed 27: 06_1048.xml
✅ Parsed 28: 06_1049.xml
✅ Parsed 29: 06_105.xml
✅ Parsed 30: 06_1050.xml
✅ Parsed 31: 06_1051.xml
✅ Parsed 32: 06_1054.xml
✅ Parsed 33: 06_1056.xml
✅ Parsed 34: 06_106.xml
✅ Parsed 35: 06_1066.xml
✅ Parsed 36: 06_1069.xml
✅ Parsed 37: 06_107.xml
✅ Parsed 38: 06_1070.xml
✅ Parsed 39: 06_1071.xml
✅ Parsed 40: 06_1072.xml
✅ Parsed 41: 06_1

In [8]:
print(cases[0])

{'file': '06_1.xml', 'case_name': 'Sharman Networks Ltd v Universal Music Australia Pty Ltd [2006] FCA 1 (5 January 2006)', 'catchphrases_list': ['"id=c0">application for leave to appeal', '"id=c1">authorisation of multiple infringements of copyright established', '"id=c2">prior sale of realty of one respondent to primary proceedings', '"id=c3">payment of substantial part of proceeds of sale to offshore company in purported repayment of loan', '"id=c4">absence of material establishing original making and purpose of loan', '"id=c5">mareva and ancillary orders made by primary judge', '"id=c6">affidavits disclosing assets sworn', '"id=c7">orders made requiring filing of further affidavits of disclosure and cross-examination of one respondent to primary proceedings on her disclosure affidavit', '"id=c8">no error in making further ancillary orders', '"id=c9">leave refused', '"id=c10">practice and procedure'], 'catchphrase_count': 11, 'sentences': 233, 'word_count': 9832, 'full_text': 'Backg

In [9]:
df.head(10)

,file,case_name,catchphrases_list,catchphrase_count,sentences,word_count,full_text,year,court_code,court_level,keyword_count,recency_years
0,06_1.xml,Sharman Networks Ltd v Universal Music Austral...,"[""id=c0"">application for leave to appeal, ""id=...",11,233,9832,Background to the current application \n \n1 T...,2006,FCA,2,1237,19
1,06_100.xml,Lawrance v Human Rights and Equal Opportunity ...,"[""id=c0"">no point of principle, ""id=c1"">admini...",2,52,1418,1 These are two applications for orders of rev...,2006,FCA,2,203,19
2,06_1001.xml,Citrus Queensland Pty Ltd v Sunstate Orchards ...,"[""id=c0"">discovery, ""id=c1"">whether inclusion ...",9,442,13466,1 I have before me two notices of motion both ...,2006,FCA,2,1785,19
3,06_1004.xml,Martech International Pty Ltd v Energy World C...,"[""id=c0"">variation, ""id=c1"">termination, ""id=c...",38,2024,47308,Introduction \n \n1 In 1985 Mr Fletcher Brand ...,2006,FCA,2,4654,19
4,06_1005.xml,Commissioner of Taxation v Milne (with Corrige...,"[""id=c0"">context of liability to income tax, ""...",5,239,13145,Context to the present application by way of a...,2006,FCA,2,1556,19
5,06_1006.xml,SZCCX v Minister for Immigration & Multicultur...,"[""id=c0"">no point in principle, ""id=c1"">migrat...",2,32,1427,1 This is an appeal against the judgment of Fe...,2006,FCA,2,151,19
6,06_1015.xml,Douglas v Queensland [2006] FCA 1015 (8 August...,"[""id=c0"">vacation of trial date, ""id=c1"">incon...",4,21,821,1 The notice of motion filed by the applicants...,2006,FCA,2,85,19
7,06_1017.xml,Regional Publishers Pty Limited v Elkington [2...,"[""id=c0"">compulsory acquisition of shares, ""id...",7,277,7658,"1 The first and second defendants, Gordon Brad...",2006,FCA,2,926,19
8,06_1018.xml,SZFBU v Minister for Immigration and Multicult...,"[""id=c0"">application for extension of time wit...",2,34,1125,1 This is an application for extension of time...,2006,FCA,2,95,19
9,06_1019.xml,SZCXK v Minister for Immigration & Multicultur...,[],0,0,0,,2006,FCA,2,0,19


In [10]:
# ------------------------------
# Cell 4 - Heuristic Scoring & Ranking
# ------------------------------
import math
import numpy as np

# tunable weights (adjustable for experimentation / hackathon pitch)
ALPHA = 2.0   # weight for court_level
BETA  = 1.5   # weight for log1p(catchphrase_count)
GAMMA = 1.2   # weight for log1p(sentence_count)
KAPPA = 0.8   # weight for keyword_count
DELTA = 0.05  # weight for recency penalty (per year)

def heuristic_influence(row):
    score = (
        ALPHA * row["court_level"]
        + BETA * math.log1p(row["catchphrase_count"])
        + GAMMA * math.log1p(row["sentences"])
        + KAPPA * row["keyword_count"]
        - DELTA * row["recency_years"]
    )
    return float(score)

# compute heuristic score
df["heuristic_score"] = df.apply(heuristic_influence, axis=1)

# rank cases by heuristic score
df = df.sort_values("heuristic_score", ascending=False).reset_index(drop=True)
df["heuristic_rank"] = df["heuristic_score"].rank(method="first", ascending=False).astype(int)

# save a trimmed CSV for ranking output
df_for_csv = df[[
    "case_name", "year", "court_code", "catchphrases_list",
    "sentences", "word_count", "heuristic_score", "heuristic_rank","recency_years"
]]
df_for_csv.to_csv("heuristic_ranking.csv", index=False, encoding="utf-8")

print("✅ Heuristic ranking complete, CSV saved as heuristic_ranking.csv")

# show top 10
df_for_csv.head(10)


✅ Heuristic ranking complete, CSV saved as heuristic_ranking.csv


,case_name,year,court_code,catchphrases_list,sentences,word_count,heuristic_score,heuristic_rank,recency_years
0,Seven Network Limited v News Limited [2007] FC...,2007,FCA,"[""id=c0"">retail pay television provider (' fox...",14643,427735,41816.385365,1,18
1,University of Western Australia v Gray (No 20)...,2008,FCA,"[""id=c0"">patents, ""id=c1"">inventions, ""id=c2"">...",8974,212915,18277.710665,2,17
2,Bennell v Western Australia [2006] FCA 1243 (1...,2006,FCA,"[""id=c0"">overlapping claimant applications in ...",5674,131848,11771.884666,3,19
3,Jango v Northern Territory of Australia (inclu...,2006,FCA,"[""id=c0"">compensation claim, ""id=c1"">applicant...",3648,108238,11645.663008,4,19
4,Australian Competition & Consumer Commission v...,2007,FCA,"[""id=c0"">price-fixing, ""id=c1"">arrangements or...",5501,135232,10557.402521,5,18
5,Risk v Northern Territory of Australia (with C...,2006,FCA,"[""id=c0"">application for determination of nati...",4008,110177,9756.292701,6,19
6,Australian Securities & Investments Commission...,2009,FCA,"[""id=c0"">misleading or deceptive conduct under...",3204,99992,9371.134385,7,16
7,Fubilan Catering Services Limited v Compass Gr...,2007,FCA,"[""id=c0"">unconscionable conduct, ""id=c1"">speci...",4601,96025,8761.547961,8,18
8,Australian Competition and Consumer Commission...,2006,FCA,"[""id=c0"">replaces redacted version see orders ...",3460,90984,8533.677094,9,19
9,A & D Douglas Pty Ltd ACN 008 404 180 v Lawyer...,2006,FCA,"[""id=c0"">solicitor's private mortgage lending ...",4918,93854,8185.107412,10,19


In [11]:

import pandas as pd


print("🔍 Null Value Summary:\n")
print(df_for_csv.isnull().sum())



🔍 Null Value Summary:

case_name            0
year                 0
court_code           0
catchphrases_list    0
sentences            0
word_count           0
heuristic_score      0
heuristic_rank       0
recency_years        0
dtype: int64


In [12]:

final_df= df_for_csv[(df_for_csv["sentences"] > 0) & (df_for_csv["word_count"] > 0) & (df_for_csv["recency_years"] > 0) ]


final_df = final_df[final_df["catchphrases_list"].apply(lambda x: len(x) > 0)]


final_df = final_df.reset_index(drop=True)


print(f" Cleaned DataFrame: {len(df_for_csv)} → {len(final_df)} rows")


 Cleaned DataFrame: 4192 → 3890 rows


In [13]:
final_df.head(10)

,case_name,year,court_code,catchphrases_list,sentences,word_count,heuristic_score,heuristic_rank,recency_years
0,Seven Network Limited v News Limited [2007] FC...,2007,FCA,"[""id=c0"">retail pay television provider (' fox...",14643,427735,41816.385365,1,18
1,University of Western Australia v Gray (No 20)...,2008,FCA,"[""id=c0"">patents, ""id=c1"">inventions, ""id=c2"">...",8974,212915,18277.710665,2,17
2,Bennell v Western Australia [2006] FCA 1243 (1...,2006,FCA,"[""id=c0"">overlapping claimant applications in ...",5674,131848,11771.884666,3,19
3,Jango v Northern Territory of Australia (inclu...,2006,FCA,"[""id=c0"">compensation claim, ""id=c1"">applicant...",3648,108238,11645.663008,4,19
4,Australian Competition & Consumer Commission v...,2007,FCA,"[""id=c0"">price-fixing, ""id=c1"">arrangements or...",5501,135232,10557.402521,5,18
5,Risk v Northern Territory of Australia (with C...,2006,FCA,"[""id=c0"">application for determination of nati...",4008,110177,9756.292701,6,19
6,Australian Securities & Investments Commission...,2009,FCA,"[""id=c0"">misleading or deceptive conduct under...",3204,99992,9371.134385,7,16
7,Fubilan Catering Services Limited v Compass Gr...,2007,FCA,"[""id=c0"">unconscionable conduct, ""id=c1"">speci...",4601,96025,8761.547961,8,18
8,Australian Competition and Consumer Commission...,2006,FCA,"[""id=c0"">replaces redacted version see orders ...",3460,90984,8533.677094,9,19
9,A & D Douglas Pty Ltd ACN 008 404 180 v Lawyer...,2006,FCA,"[""id=c0"">solicitor's private mortgage lending ...",4918,93854,8185.107412,10,19


In [20]:
from transformers import BertTokenizer, BertModel

# Initialize BERT Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Save the tokenizer and model using Hugging Face methods
tokenizer.save_pretrained('./bert_tokenizer')
model.save_pretrained('./bert_model')

print("✅ BERT Model and Tokenizer are successfully saved!")


✅ BERT Model and Tokenizer are successfully saved!


In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from transformers import BertTokenizer, BertModel
from scipy.sparse import hstack

# Step 1: Load the saved BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('./bert_tokenizer')
model = BertModel.from_pretrained('./bert_model')

# Function to extract BERT embeddings
def bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Using the mean of token embeddings
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# Step 2: Apply BERT embeddings to 'case_name' and 'catchphrases_list'
final_df['bert_case_name'] = final_df['case_name'].apply(bert_embeddings)
final_df['bert_catchphrases'] = final_df['catchphrases_list'].apply(lambda x: ' '.join(x)).apply(bert_embeddings)

# Convert BERT embeddings to a numpy array for training
X_bert_case_name = np.stack(final_df['bert_case_name'].values)
X_bert_catchphrases = np.stack(final_df['bert_catchphrases'].values)

# Step 3: Combine with other numeric features (e.g., 'year', 'sentences', 'word_count', etc.)
X_numeric = final_df[["year", "sentences", "word_count", "recency_years"]].values

# Combine numeric and BERT features into a single feature matrix
X_combined_bert = np.hstack([X_numeric, X_bert_case_name, X_bert_catchphrases])

# Step 4: Labels (heuristic score to predict)
y = final_df['heuristic_score']

# Step 5: Train-Test Split (while preserving the original index for later use)
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X_combined_bert, y, final_df.index, test_size=0.2, random_state=42
)

# Step 6: Train Random Forest Model with BERT features
rf_bert = RandomForestRegressor(n_estimators=200, random_state=42)
rf_bert.fit(X_train, y_train)

# Step 7: Predictions
y_pred_bert = rf_bert.predict(X_test)

# Step 8: Evaluation
print("✅ BERT + RandomForest Evaluation:")
print(f"R² Score: {r2_score(y_test, y_pred_bert)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_bert)}")

# Step 9: Create a DataFrame to hold the case names and the predicted scores
ranking_bert = pd.DataFrame({
    'case_name': final_df.iloc[test_indices]['case_name'],  # Use original indices to get the case names
    'true_score': y_test,  # True heuristic scores
    'predicted_score': y_pred_bert  # Predicted heuristic scores from BERT model
})

# Step 10: Sort the cases by predicted scores (in descending order for higher scores first)
ranking_bert = ranking_bert.sort_values(by="predicted_score", ascending=False).reset_index(drop=True)

# Step 11: Display the top 10 ranked cases
print("✅ Top 10 Cases Ranked by Predicted Heuristic Score (BERT):")
print(ranking_bert.head(10))

# Step 12: Compare with true heuristic rank
ranking_bert['true_rank'] = ranking_bert['true_score'].rank(ascending=False, method="first").astype(int)
print("✅ Top 10 Cases with True Ranks and Predicted Ranks:")
print(ranking_bert[['case_name', 'true_rank', 'predicted_score']].head(10))


✅ BERT + RandomForest Evaluation:
R² Score: 0.5489547378297898
MAE: 94.92181820931519
✅ Top 10 Cases Ranked by Predicted Heuristic Score (BERT):
                                           case_name    true_score  \
0  Seven Network Limited v News Limited [2007] FC...  41816.385365   
1  Australian Competition and Consumer Commission...   8533.677094   
2  Pacific National (ACT) Limited (ACN 48 052 134...   7098.222386   
3  Bropho v State of Western Australia (Corrigend...   5937.880214   
4  Alphapharm Pty Ltd v H Lundbeck A/S (includes ...   6740.932886   
5  Biodiesel Producers Limited (ACN 099 165 876) ...   4822.810606   
6  North East Equity Pty Ltd v Proud Nominees Pty...   4527.824461   
7  Australian Competition and Consumer Commission...   4245.985849   
8  Martech International Pty Ltd v Energy World C...   3740.881332   
9  Wilton & Cumberland v Coal & Allied Operations...   4503.382474   

   predicted_score  
0     10540.751306  
1      9548.003813  
2      8943.594949  
